In [1]:
import tensorflow
physical_devices = tensorflow.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [5]:
img_width,img_height =224,224
input_shape=(img_width,img_height,3)

### Model Creation

In [6]:
model = keras.models.Sequential([
 keras.layers.Conv2D(32, (5, 5),input_shape=input_shape,activation='relu',name="conv2d_1"),
 keras.layers.MaxPool2D(pool_size=(3, 3),name="max_pooling2d_1"),
 keras.layers.Conv2D(32, (3, 3),activation='relu',name="conv2d_2"),
 keras.layers.MaxPool2D(pool_size=(2, 2),name="max_pooling2d_2"),
 keras.layers.Conv2D(64, (3, 3),activation='relu',name="conv2d_3"),
 keras.layers.MaxPool2D(pool_size=(2, 2),name="max_pooling2d_3"),   
 keras.layers.Flatten(name="flatten_1"),
 keras.layers.Dense(512,activation='relu'),
 keras.layers.Dropout(0.25),
 keras.layers.Dense(256,activation='relu'),
 keras.layers.Dense(64,activation='relu'),  
 keras.layers.Dense(38,activation='softmax')
  ]
)




### Loading Weights To The Model

In [7]:
model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32
base_dir = "dataset"

training_set = train_datagen.flow_from_directory(base_dir+'/train',
                                                 target_size=(224, 224),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

valid_set = valid_datagen.flow_from_directory(base_dir+'/valid',
                                            target_size=(224, 224),
                                            batch_size=batch_size,
                                            class_mode='categorical')


Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In [9]:
train_num = training_set.samples
valid_num = valid_set.samples

In [10]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)



In [11]:

#fitting images to CNN
history = model.fit_generator(training_set,
                         steps_per_epoch=train_num//batch_size
                              ,
                         validation_data=valid_set,
                         epochs=25,
                         validation_steps= valid_num//batch_size,callbacks=[reduce_lr],
                            shuffle=True)
    

P:\Conda\envs\tf_24\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
2196/2196 [==============================] - 1409s 638ms/step - loss: 3.3677 - accuracy: 0.0928 - val_loss: 2.1967 - val_accuracy: 0.3488
Epoch 2/25
2196/2196 [==============================] - 723s 329ms/step - loss: 1.9698 - accuracy: 0.4141 - val_loss: 0.9505 - val_accuracy: 0.7016
Epoch 3/25
2196/2196 [==============================] - 735s 335ms/step - loss: 1.2123 - accuracy: 0.6190 - val_loss: 0.7042 - val_accuracy: 0.7723
Epoch 4/25
2196/2196 [==============================] - 721s 328ms/step - loss: 0.8724 - accuracy: 0.7238 - val_loss: 0.6213 - val_accuracy: 0.7873
Epoch 5/25
2196/2196 [==============================] - 724s 330ms/step - loss: 0.7183 - accuracy: 0.7685 - val_loss: 0.4678 - val_accuracy: 0.8466
Epoch 6/25
2196/2196 [==============================] - 759s 345ms/step - loss: 0.5981 - accuracy: 0.8070 - val_loss: 0.2890 - val_accuracy: 0.9066
Epoch 7/25
2196/2196 [==============================] - 784s 357ms/step - loss: 0.5115 - accuracy: 0.8334 - val

In [12]:
path="CNNBiasa2.hdf5"
model.save(path)